### Feature Engineering
##### It's a simple step, extract all edges in the train.txt document. Then save all edges line by line in edges.txt


In [1]:
# Candidate1: self-circles only exists for non-existence nodes in test-public.

import pandas as pd

nodes = []
with open('data/train.txt', 'r') as f, open('data/edges.txt', 'w') as o, open('data/test-public.csv', 'r') as t:
    training_data = f.readlines()
    for line in training_data:
        l = line.split() # each line ['str1', 'str2'] 
        for i in range(len(l)):
            nodes.append(l[i])
            for j in range(len(l)):
                if i!=j:
                    o.write(f'{l[i]} {l[j]}\n')
    setOfNodes = set(nodes)

    count = 0

    test_data = t.readlines()
    test_data = test_data[1:]

    for line in test_data:
        line = line[:len(line) -1]
        l = line.split(',')
        for i in range(1, len(l)):
            if l[i] not in setOfNodes:
                o.write(f'{l[i]} {l[i]}\n')
                count+=1
    print('Added', count, ' self-circles.')            


Added 54  self-circles.


In [2]:
# Check the size of results. 

!wc -l data/edges.txt

   59294 data/edges.txt


In [6]:
# Candidate2: No duplicated edges, no test-public consideration
nodes = []
edges = []
with open('data/train.txt', 'r') as f, open('data/edges_no_duplicate.txt', 'w') as o:
    training_data = f.readlines()
    for line in training_data:
        l = line.split() # each line ['str1', 'str2'] 
        for i in range(len(l)):
            nodes.append(l[i])
            for j in range(len(l)):
                if i!=j:
                    #o.write(f'{l[i]} {l[j]}\n')
                    edges.append((int(l[i]), int(l[j])))

    setOfNodes = set(nodes)
    setOfEdges = set(edges)
    print('nodes without duplicate: ', len(setOfNodes))
    print('edges without duplicate: ', len(setOfEdges))
          
    no_dup_edges = list(setOfEdges)
    for elem in no_dup_edges:
        o.write(f'{elem[0]} {elem[1]}\n')

nodes without duplicate:  3767
edges without duplicate:  32070


In [ ]:
!wc -l data/edges_no_duplicate.txt

32070 data/edges_no_duplicate.txt


In [3]:
# Split validation set
import numpy as np

def link_prediction_split(graph_file, files, portions):
    """
    Divide a normal graph into a train split and several test splits for link prediction use.
    Each test split contains half true and half false edges.

    Parameters:
        graph_file (str): graph file
        files (list of str): file names,
            the first file is treated as train file
        portions (list of float): split portions
    """
    assert len(files) == len(portions)
    print(f"splitting graph {graph_file} into {', '.join([f for f in files])}")
    np.random.seed(1024)

    nodes = set()
    edges = set()
    portions = np.cumsum(portions, dtype=np.float32) / np.sum(portions)
    files = [open(file, "w") for file in files]
    num_edges = [0] * len(files)
    with open(graph_file, "r") as fin:
        for line in fin:
            u, v = line.split()[:2]
            nodes.update([u, v])
            edges.add((u, v))
            i = np.searchsorted(portions, np.random.rand())
            if i == 0:
                files[i].write(line)
            else:
                files[i].write(f"{u}\t{v}\t1\n")
            num_edges[i] += 1

    nodes = list(nodes)
    for file, num_edge in zip(files[1:], num_edges[1:]):
        for _ in range(num_edge):
            valid = False
            while not valid:
                u = nodes[int(np.random.rand() * len(nodes))]
                v = nodes[int(np.random.rand() * len(nodes))]
                valid = u != v and (u, v) not in edges and (v, u) not in edges
            file.write(f"{u}\t{v}\t0\n")
    for file in files:
        file.close()

link_prediction_split('data/edges.txt', ['train8.txt', 'test2.txt'], [8, 2])

splitting graph data/edges.txt into train8.txt, test2.txt


### NOTES On Colab
#### When you run your code on colab you need to load the goole's virtual drive. Run the following blocks to load drive and 'cd' to the current path.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd drive/MyDrive/Graph_Link_Prediction/graphvite/

/content/drive/MyDrive/Graph_Link_Prediction/graphvite
